# Define seed points

This notebook is based on the notebook of Michael Szell "02_prepare_pois.ipynb" that you can find here:

https://github.com/mszell/bikenwgrowth

### Parameters

In [ ]:
#debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Download and wrangle data

In [ ]:
# Load all carall graphs in OSMNX format
parameterinfo = osmnxparameters['carall']

for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
    print(placeid + ": Loading location polygon and carall graph")
    
    if placeinfo["nominatimstring"] != '':
        location = ox.geocoder.geocode_to_gdf(placeinfo["nominatimstring"])
        location = fill_holes(extract_relevant_polygon(placeid, shapely.geometry.shape(location['geometry'][0])))
    else:
        # https://gis.stackexchange.com/questions/113799/how-to-read-a-shapefile-in-python
        shp = fiona.open(PATH["data"] + placeid + "/" + placeid + ".shp")
        first = next(iter(shp))
        location = shapely.geometry.shape(first['geometry'])
    
    
G_biketrackcarall = csv_to_ox(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')
G_biketrackcarall.graph["crs"] = 'epsg:4326' # needed for OSMNX's graph_to_gdfs in utils_graph.py
G_biketrackcarall_simplified = csv_to_ox(PATH["data"] + placeid + "/", placeid, 'biketrackcarall_simplified')
G_biketrackcarall_simplified.graph["crs"] = 'epsg:4326' # needed for OSMNX's graph_to_gdfs in utils_graph.py
    
# Load biketrack network (iGraph format)
G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')

In [ ]:
# Load pop data (Turin 2020)
pop_shp = gpd.read_file(PATH["data"] + placeid + "/" +'popTorino2020')

# all the seed points located in regions with pop density < min_density will be removed

min_density = 1185

low_pop_density = pop_shp[pop_shp['dens_totpo'] < min_density]

# remove from the low_pop_density df the city-centre regions (parks)

low_pop_density = low_pop_density.drop([1,9,37])

Using code from: https://github.com/gboeing/osmnx-examples/blob/v0.11/notebooks/17-street-network-orientations.ipynb

In [ ]:
delta = 300 # delta param
  
# FIRST, determine the most common bearing, for the best grid orientation
G = G_biketrackcarall
bearings = {}    
# calculate edge bearings
Gu = ox.bearing.add_edge_bearings(ox.get_undirected(G))

# weight bearings by length (meters)
city_bearings = []
for u, v, k, d in Gu.edges(keys = True, data = True):
    city_bearings.extend([d['bearing']] * int(d['length']))
b = pd.Series(city_bearings)
bearings = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop = 'True')

bins = np.arange(bearingbins + 1) * 360 / bearingbins
count = count_and_merge(bearingbins, bearings)
principalbearing = bins[np.where(count == max(count))][0]


# SECOND, construct the grid
G = G_biketrackcarall_simplified

# 1) Get lat lon window, with buffer for snapping outside seed points
# https://stackoverflow.com/questions/7477003/calculating-new-longitude-latitude-from-old-n-meters
buf = max(((2*snapthreshold) / 6378000) * (180 / math.pi), 
        ((2*snapthreshold) / 6378000) * (180 / math.pi) / math.cos(location.centroid.y * math.pi/180)
        )
cities[placeid]["bbox"] = location.buffer(buf).bounds

# 2) Generate abstract grid points in window
# https://stackoverflow.com/questions/40342355/how-can-i-generate-a-regular-geographic-grid-using-python
# Set up projections
p_ll = pyproj.Proj('+proj=longlat +datum=WGS84')
aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
p_mt = pyproj.Proj(aeqd_proj.format(lat=location.centroid.y, lon=location.centroid.x)) # equidistant projection

# Make the area larger to account for tilted grid
deltax = cities[placeid]["bbox"][2] - cities[placeid]["bbox"][0]
deltay = cities[placeid]["bbox"][3] - cities[placeid]["bbox"][1]
enlargefactor = 10

# Create corners of rectangle to be transformed to a grid
sw = shapely.geometry.Point((cities[placeid]["bbox"][0], cities[placeid]["bbox"][1]))
ne = shapely.geometry.Point((cities[placeid]["bbox"][2]+enlargefactor*deltax, cities[placeid]["bbox"][3]+enlargefactor*deltay))
    
# Project corners to target projection
transformed_sw = pyproj.transform(p_ll, p_mt, sw.x, sw.y) # Transform NW point to equidistant
transformed_ne = pyproj.transform(p_ll, p_mt, ne.x, ne.y) # .. same for SE

# Iterate over 2D area
principalbearing = principalbearing % 90 # Take it modulo 90 because it will be a square grid
if principalbearing > 45:
    principalbearing -= 90 # Make bearing fall btw -45 and 45

xcoords = np.arange(transformed_sw[0], transformed_ne[0], gridl)
ycoords = np.arange(transformed_sw[1], transformed_ne[1], gridl)
xsize =  xcoords.size
ysize = ycoords.size
xcoords = np.tile(xcoords, ysize)
ycoords = np.repeat(ycoords, xsize)
gridpoints=[(x, y) for x, y in zip(xcoords, ycoords)]
new_points = rotate_grid(gridpoints, origin = transformed_sw, degrees = principalbearing)
    
# https://stackoverflow.com/questions/42459068/projecting-a-numpy-array-of-coordinates-using-pyproj
fx, fy = pyproj.transform(p_mt, p_ll, new_points[:,0], new_points[:,1])
gridpoints = np.dstack([fx, fy])[0]
if principalbearing >=0:
    # If we rotated right, we need to shift everything to the left
    gridpoints[:,0] -= 0.4*enlargefactor*deltax*math.sin(np.deg2rad(principalbearing))
else:
    # If we rotated left, we need to shift everything down and to the right
    gridpoints[:,0] += 0.4*enlargefactor*deltax*math.sin(np.deg2rad(principalbearing))
    gridpoints[:,1] -= 0.4*enlargefactor*deltay

# Cut back to bounding box
mask = (gridpoints[:,0] >= cities[placeid]["bbox"][0]) & (gridpoints[:,0] <= cities[placeid]["bbox"][2]) & (gridpoints[:,1] >= cities[placeid]["bbox"][1]) & (gridpoints[:,1] <= cities[placeid]["bbox"][3])
gridpoints_cut = gridpoints[mask]

# 3) Snap grid points to map

nnids = set() # set of nodes (seed points)
    
# add to the seed points the nodes of the existing bike net
existing_nnids = G_biketrack.vs['id']        
    
for n in existing_nnids:    
    add_point = True
        
        
    if G.has_node(n):
            
        # check that the POI is not in a low density region
        for kkk in range(len(low_pop_density)):
            if Point(G.nodes[n]["x"],G.nodes[n]["y"]).within(low_pop_density['geometry'].iloc[kkk]):
                add_point = False
                break
            
        # check that the seed points distance is <= delta for each pair of seed points; if not, remove the POI
        if add_point:
            for jj in nnids:
                if haversine((G.nodes[jj]["y"], G.nodes[jj]["x"]), (G.nodes[n]["y"], G.nodes[n]["x"]), unit="m") <= delta:
                    add_point = False
                    break
                    
    else:
        add_point = False
        
    # add the node to the seed points set if no exception occurred
    if add_point:
        nnids.add(n)
        
# add to the seed points the grid nodes
    
for g in gridpoints_cut:
    n = ox.distance.get_nearest_node(G, [g[1], g[0]])
    add_point = True
    if n not in nnids and haversine((g[1], g[0]), (G.nodes[n]["y"], G.nodes[n]["x"]), unit="m") <= snapthreshold:
            
        # check that the seed point is not in a low density region
        for kkk in range(len(low_pop_density)):
            if Point(G.nodes[n]["x"],G.nodes[n]["y"]).within(low_pop_density['geometry'].iloc[kkk]):
                add_point = False
                break
            
        # check that the new seed point is at least delta m far the other seed points
        if add_point:
            for jj in nnids:
                if haversine((G.nodes[jj]["y"], G.nodes[jj]["x"]), (G.nodes[n]["y"], G.nodes[n]["x"]), unit="m") <= delta:
                    add_point = False
                    break
                    
        # add the node to the seed points set if no exception occurred        
        if add_point:
            nnids.add(n)

# save the nodes for the seed points
with open(PATH["data"] + placeid + "/" + placeid + '_seed-points_delta' + str(delta) + '.csv', 'w') as f:
    for item in nnids:
        f.write("%s\n" % item)